## Importing existing trajectory data

In many cases, some trajectory data already exists before running an adaptive simulation. It is thus most efficiently to import this data into the framework. This works in principle by creating `Trajectory` objects and adding them to the `Project`. Since all of the trajectory-related data however is stored in the `Engine` object that generated it, this needs to be created as well.

### Imports

In [ ]:
import sys, os

In [ ]:
from adaptivemd import Project

Let's open our `test` project by its name. If you completed the previous example this should all work out of the box.

In [ ]:
project = Project('tutorial')

Open all connections to the `MongoDB` and `Session` so we can get started.

### Create an import `Engine`

In [ ]:
from adaptivemd import Trajectory
import time

In [ ]:
pdb_file = File('file://init.pdb').named('initial_pdb').load()

Since it is not desired to expand the trajectories at this point, system and integrator files are not given. In principle, if compatible restart files are available, one could create a complete engine and expand existing trajectories.

In [ ]:
import_engine = OpenMMEngine(pdb_file=pdb_file,
                             system_file=None,
                             integrator_file=None,
                             args=None
                             ).named('openmm-import')

Now, to use the same `Modeller` as for the trajectories generated with `AdaptiveMD`, we build compatible output types. This means, they should contain the original file names with the respective strides and be named accordly. 

In [ ]:
import_engine.add_output_type('master', 'old-file-name-full.dcd', 
                              stride=stride_full)
import_engine.add_output_type('protein', 'old-file-name-protein.dcd', 
                              stride=stride_prot, 
                              selection='protein')

## Initialize `Trajectory` objects
To add the actual file paths, `Trajectory` objects have to be initialized. 
- `Trajectory` locations are folders, not files, and end with '/'.
- `frame` can be None if the initial frame is not known.
- `length` as defined by the engine time step, not by the output/save rate of an output type.
- `engine`: import engine defined above.

The example below uses a list of trajectory folders to import, `existing_trajectory_paths`. Note that when adding this path with the `shared://` prefix, it must be a relative path in the root shared cluster file system. For absolute paths, use `worker://` instead. The trajectory lengths are known and stored in `existing_trajectory_lengths`.

The `created` variable has to be set a creation time in order to let the database know the trajectory already exists. In the example below, the (arbitrary) import time is used.

In [ ]:
trajs = []
for traj_path, traj_length in zip(existing_trajectory_paths, 
                                  existing_trajectory_lengths):
    traj = Trajectory('shared://' + traj_path,
                      frame=None,
                      length=traj_length,
                      engine=import_engine)
    traj.created = time.time()
    trajs.append(traj)

#### Add the trajectories to the project

In [ ]:
map(project.files.add, trajs)

Let's check if the trajectories have been added:

In [ ]:
len(project.trajectories)

In [ ]:
project.close()